# Time series analysis and visualisation

Heavily inspired by Jennifer Walker's tutorial: https://www.dataquest.io/blog/tutorial-time-series-analysis-with-pandas/ 

In [ ]:
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
# Use seaborn style defaults and set the default figure size
sns.set_theme(rc={'figure.figsize':(11, 4)})

#### The Open Power Systems Dataset (OPSD)

In this demo, we will be using the daily time series of Open Power System Data (OPSD) for Germany. Germany has been rapidly expanding its renewable energy production in the 2000s-2010s. The data set includes country-wide totals of electricity consumption, wind power production, and solar power production for the years 2006-2017. You can read more about the data here: https://www.kaggle.com/mvianna10/germany-electricity-power-for-20062017.

We will download the CSV dataset from GitHub: https://raw.githubusercontent.com/jenfly/opsd/master/opsd_germany_daily.csv

Electricity production and consumption are reported as daily totals in gigawatt-hours (GWh). The columns of the data file are:

* **Date** — The date (yyyy-mm-dd format)
* **Consumption** — Electricity consumption in GWh
* **Wind** — Wind power production in GWh
* **Solar** — Solar power production in GWh
* **Wind+Solar** — Sum of wind and solar power production in GWh

We will explore how electricity production and consumption in Germany have changed over time, using pandas time series tools to answer questions such as:

* When is electricity consumption typically highest and lowest?
* How do wind and solar power production vary with seasons of the year?
* What are the long-term trends in electricity consumption, solar power, and wind power?
* How do wind and solar power production compare with electricity consumption, and how has this ratio changed over time?

In [ ]:
opsd_daily = pd.read_csv(
    'https://raw.githubusercontent.com/jenfly/opsd/master/opsd_germany_daily.csv'
)
opsd_daily.shape

In [ ]:
opsd_daily.head()

In [ ]:
opsd_daily.tail()

### Time series data structures

Before we dive into the OPSD data, let’s briefly introduce the main pandas data structures for working with dates and times. In pandas, a single point in time is represented as a Timestamp. We can use the to_datetime() function to create Timestamps from strings in a wide variety of date/time formats. 

In [ ]:
opsd_daily.info()

In [ ]:
opsd_daily["Date"] = pd.to_datetime(opsd_daily["Date"])

In [ ]:
opsd_daily.info()

Now that the `Date` column is of the `datetime64` type, we can set it as the DataFrame’s index.

In [ ]:
opsd_daily = opsd_daily.set_index('Date')
opsd_daily.sample(5)

In [ ]:
opsd_daily.index

In [ ]:
# As an alternative, we could set the "Date" columns as datetime64
# and set it as an index
opsd_daily = pd.read_csv(
    'https://raw.githubusercontent.com/jenfly/opsd/master/opsd_germany_daily.csv',
    index_col=0,
    parse_dates=True
)

In [ ]:
opsd_daily.sample(5)

In [ ]:
opsd_daily.info()

In [ ]:
# Add columns with year, month, and weekday name
opsd_daily['Year'] = opsd_daily.index.year
opsd_daily['Month'] = opsd_daily.index.month
opsd_daily['Weekday Name'] = opsd_daily.index.day_name()
# Display a random sampling of 5 rows
opsd_daily.sample(5)

### Time-based indexing

This works the same as regular indexing and slicing:

In [ ]:
opsd_daily.loc['2017-07-07']

In [ ]:
opsd_daily.loc['2012-01-01':'2013-07-01']

## Partial string indexing

Another very handy feature of pandas time series is partial-string indexing, where we can select all date/times which partially match a given string.

You could use it to get entire months or years of data

In [ ]:
# get the whole of July 2015
opsd_daily.loc['2015-07']

### Time series data visualisation

In [ ]:
opsd_daily['Consumption'].plot(linewidth=0.5);

The `plot()` method has chosen pretty good tick locations (every two years) and labels (the years) for the x-axis

In [ ]:
cols_plot = ['Consumption', 'Solar', 'Wind']
axes = opsd_daily[cols_plot].plot(
    marker='.',
    alpha=0.5,
    linestyle='None',
    figsize=(11, 9),
    subplots=True
)
for ax in axes:
    ax.set_ylabel('Daily Totals (GWh)')

What kind of interesting patterns do you see visualising the data?

### Seasonality

All three time series clearly exhibit periodicity—often referred to as *seasonality* in time series analysis—in which a pattern repeats again and again at regular time intervals.

In [ ]:
ax = opsd_daily.loc['2017', 'Consumption'].plot()
ax.set_ylabel('Daily Consumption (GWh)');

We can see that there are weekly oscillations.

In [ ]:
ax = opsd_daily.loc['2017-06':'2017-07', 'Consumption'].plot(
    marker='o',
    linestyle='-'
)
ax.set_ylabel('Daily Consumption (GWh)');

To better visualize the weekly seasonality in electricity consumption in the plot above, it would be nice to have vertical gridlines on a weekly time scale (instead of on the first day of each month). We can customize our plot with matplotlib.dates, so let’s import that module.

In [ ]:
import matplotlib.dates as mdates

Because date/time ticks are handled a bit differently in matplotlib.dates compared with the DataFrame’s `plot()` method, let’s create the plot directly in matplotlib. Then we use `mdates.WeekdayLocator()` and `mdates.MONDAY` to set the x-axis ticks to the first Monday of each week. We also use `mdates.DateFormatter()` to improve the formatting of the tick labels, using the format codes we saw earlier.

In [ ]:
fig, ax = plt.subplots()
ax.plot(
    opsd_daily.loc['2017-06':'2017-07', 'Consumption'],
    marker='o',
    linestyle='-'
)
ax.set_ylabel('Daily Consumption (GWh)')
ax.set_title('June-July 2017 Electricity Consumption')
# Set x-axis major ticks to weekly interval, on Mondays
ax.xaxis.set_major_locator(mdates.WeekdayLocator(byweekday=mdates.MONDAY))
# Format x-tick labels as 3-letter month name and day number
ax.xaxis.set_major_formatter(mdates.DateFormatter('%b %d'));

Now we have vertical gridlines and nicely formatted tick labels on each Monday, so we can easily tell which days are weekdays and weekends.

### Seasonality

In [ ]:
fig, axes = plt.subplots(3, 1, figsize=(11, 10), sharex=True)
for name, ax in zip(['Consumption', 'Solar', 'Wind'], axes):
    sns.boxplot(data=opsd_daily, x='Month', y=name, ax=ax)
ax.set_ylabel('GWh')
ax.set_title(name)
# Remove the automatic x-axis label from all but the bottom subplot
if ax != axes[-1]:
    ax.set_xlabel('')

In [ ]:
sns.boxplot(data=opsd_daily, x='Weekday Name', y='Consumption');

### Frequencies and frequency analysis

When the data points of a time series are uniformly spaced in time (e.g., hourly, daily, monthly, etc.), the time series can be associated with a frequency in pandas. For example, let’s use the `date_range()` function to create a sequence of uniformly spaced dates from `2017-03-10` through `2017-03-15` at daily frequency.

In [ ]:
# date_range(start, end, freq). For frequency strings see https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html#timeseries-offset-aliases
# for a list of frequency aliases. "D" = daily
pd.date_range('2017-03-10', '2017-03-15', freq='D')

The resulting `DatetimeIndex` object has an attribute `freq` with a value of 'D', indicating daily frequency. Available frequencies in pandas include hourly ('H'), calendar daily ('D'), business daily ('B'), weekly ('W'), monthly ('M'), quarterly ('Q'), annual ('A'), and many others. Frequencies can also be specified as multiples of any of the base frequencies, for example '5D' for every five days.

As another example, let’s create a date range at hourly frequency, specifying the start date and number of periods, instead of the start date and end date.

In [ ]:
pd.date_range('2004-09-20', periods=8, freq='H')

Let’s take another look at the DatetimeIndex of our `opsd_daily` time series.

In [ ]:
opsd_daily.index


At the moment it has `freq` set to `None`. We did not explicitly set a freq when we loaded it from the CSV file.

If we know that our data should be at a specific frequency, we can use the DataFrame’s `asfreq()` method to assign a frequency. If any date/times are missing in the data, new rows will be added for those date/times, which are either empty (`NaN`), or filled according to a specified data filling method such as forward filling or interpolation.

To see how this works, let’s create a new DataFrame which contains only the Consumption data for Feb 3, 6, and 8, 2013.



In [ ]:
# To select an arbitrary sequence of date/time values from a pandas time series,
# we need to use a DatetimeIndex, rather than simply a list of date/time strings
times_sample = pd.to_datetime(['2013-02-03', '2013-02-06', '2013-02-08'])
# Select the specified dates and just the Consumption column
consum_sample = opsd_daily.loc[times_sample, ['Consumption']].copy()
consum_sample

Now we use the `asfreq()` method to convert the DataFrame to daily frequency, with a column for unfilled data, and a column for forward filled data.

In [ ]:
# Convert the data to daily frequency, without filling any missings
consum_freq = consum_sample.asfreq('D')
# Create a column with missings forward filled
consum_freq['Consumption - Forward Fill'] = consum_sample.asfreq('D', method='ffill')
consum_freq

In the `Consumption` column, we have the original data, with a value of NaN for any date that was missing in our consum_sample DataFrame. 

In the `Consumption - Forward Fill` column, the missings have been forward filled, meaning that the last value repeats through the missing rows until the next non-missing value occurs.

If you’re doing any time series analysis which requires uniformly spaced data without any missings, you’ll want to use `asfreq()` to convert your time series to the specified frequency and fill any missings with an appropriate method.

### Resampling

It can be useful to resample a time series data to a higher or lower frequency. Resampling to a higher frequency (**upsampling**) is not very common and often involves interpolation or other data filling method — for example, interpolating hourly weather data to 10 minute intervals for input to a scientific model. Resampling to a lower frequency (**downsampling**) can usually involve some aggregation operation — for example, computing monthly sales totals from daily data. The daily OPSD data we’re working with in this tutorial was downsampled from the original hourly time series. 

We will focus here on downsampling, exploring how it can help us analyze our OPSD data on various time scales. We use the DataFrame’s `resample()` method, which splits the `DatetimeIndex` into time bins and groups the data by time bin. The `resample()` method returns a `Resampler` object, which is somewhat similar to a pandas `GroupBy` object. We can then apply aggregation methods such as `mean()`, `median()`, `sum()`, etc., to the data group for each time bin.

Let’s try and resample the data to a weekly mean time series.

In [ ]:
# Specify the data columns we want to include (i.e. exclude Year, Month, Weekday Name)
DATA_COLUMNS = ['Consumption', 'Wind', 'Solar', 'Wind+Solar']
# Resample to weekly frequency, aggregating with mean
opsd_weekly_mean = opsd_daily[DATA_COLUMNS].resample('W').mean()
opsd_weekly_mean.tail(4)

The first row above, labelled 2017-12-10, contains the mean of all the data contained in the time bin 2017-12-10 through 2017-12-16. The second row, labelled 2017-12-17, contains the mean data for the 2006-01-08 through 2017-12-23 time bin, and so on. By default, each row of the downsampled time series is labelled with the right edge of the time bin.

By construction, our weekly time series has ~1/7 as many data points as the daily time series. We can confirm this by comparing the number of rows of the two DataFrames.

In [ ]:
print(opsd_daily.shape)
print(opsd_weekly_mean.shape)
print(opsd_daily.shape[0]/opsd_weekly_mean.shape[0])

Let’s plot the daily and weekly Wind time series together over a single six-month period to compare them.

In [ ]:
# Start and end of the date range to extract
start, end = '2017-01', '2017-06'
# Plot daily and weekly resampled time series together
fig, ax = plt.subplots()
ax.plot(opsd_daily.loc[start:end, 'Wind'],
marker='.', linestyle='-', linewidth=0.5, label='Daily')
ax.plot(opsd_weekly_mean.loc[start:end, 'Wind'],
marker='o', markersize=8, linestyle='-', label='Weekly Mean Resample')
ax.set_ylabel('Wind Production (GWh)')
_ = ax.legend()

Now let’s resample the data to monthly frequency, aggregating with sum totals instead of the mean. 

Unlike aggregating with `mean()`, which sets the output to `NaN` for any period with all missing data, the default behavior of `sum()` will return output of 0 as the sum of missing data. We use the `min_count` parameter to change this behavior.

In [ ]:
# Compute the monthly sums, setting the value to NaN for any month which has
# fewer than 28 days of data
opsd_monthly = opsd_daily[DATA_COLUMNS].resample('M').sum(min_count=28)
opsd_monthly.tail(4)

You might notice that the monthly resampled data is labelled with the end of each month (the right bin edge), whereas the weekly resampled data is labelled with the left bin edge. By default, resampled data is labelled with the right bin edge for monthly, quarterly, and annual frequencies, and with the left bin edge for all other frequencies. This behavior and various other options can be adjusted using the parameters listed in the `resample()` documentation.

Now let’s explore the monthly time series by plotting the electricity consumption as a line plot, and the wind and solar power production together as a stacked area plot.



In [ ]:
fig, ax = plt.subplots()
ax.plot(opsd_monthly['Consumption'], color='black', label='Consumption')
opsd_monthly[['Wind', 'Solar']].plot.area(ax=ax, linewidth=0)
ax.xaxis.set_major_locator(mdates.YearLocator())
ax.legend()
_ = ax.set_ylabel('Monthly Total (GWh)')

At this monthly time scale, we can clearly see the yearly seasonality in each time series, and it is also evident that electricity consumption has been fairly stable over time, while wind power production has been growing steadily, with wind + solar power comprising an increasing share of the electricity consumed.

Let’s explore this further by resampling to annual frequency and computing the ratio of `Wind+Solar` to `Consumption` for each year.

In [ ]:
# Compute the annual sums, setting the value to NaN for any year which has
# fewer than 360 days of data
opsd_annual = opsd_daily[DATA_COLUMNS].resample('A').sum(min_count=360)
# The default index of the resampled DataFrame is the last day of each year,
# ('2006-12-31', '2007-12-31', etc.) so to make life easier, set the index
# to the year component
opsd_annual = opsd_annual.set_index(opsd_annual.index.year)
opsd_annual.index.name = 'Year'
# Compute the ratio of Wind+Solar to Consumption
opsd_annual['Wind+Solar %'] = 100.0 * opsd_annual['Wind+Solar'] / opsd_annual['Consumption']
opsd_annual

In [ ]:
# Plot from 2012 onwards, because there is no solar production data in earlier years
ax = opsd_annual.loc[2012:, 'Wind+Solar %'].plot.bar(color='C0')
ax.set_ylabel('Percentage')
ax.set_ylim(0, 100)
ax.set_title('Wind + Solar Share of Annual Electricity Consumption')
_ = plt.xticks(rotation=0)

## Rolling windows

Rolling window operations are another important transformation for time series data. Similar to downsampling, rolling windows split the data into time windows and and the data in each window is aggregated with a function such as `mean()`, `median()`, `sum()`, etc. However, unlike downsampling, where the time bins do not overlap and the output is at a lower frequency than the input, rolling windows overlap and “roll” along at the same frequency as the data, so the transformed time series is at the same frequency as the original time series.

By default, all data points within a window are equally weighted in the aggregation, but this can be changed by specifying window types such as Gaussian, triangular, and others. We’ll stick with the standard equally weighted window here.

Let’s use the `rolling()` method to compute the 5-day rolling mean of our daily data. We use the `center=True` argument to label each window at its midpoint, so the rolling windows are:

* `2006-01-01` to `2006-01-05` — labelled as `2006-01-03`
* `2006-01-02` to `2006-01-06` — labelled as `2006-01-04`
and so on…

In [ ]:
opsd_5d = opsd_daily[DATA_COLUMNS].rolling(5, center=True).mean()
opsd_5d.sample(5)

In [ ]:
opsd_5d.head(5)

We can see that the first non-missing rolling mean value is on `2006-01-03`, because this is the midpoint of the first rolling window.

To visualize the differences between rolling mean and resampling, let’s update our earlier plot of January-June 2017 solar power production to include the 7-day rolling mean along with the weekly mean resampled time series and the original daily data.



In [ ]:
# compute 5-day mean
opsd_5d_mean = opsd_daily[DATA_COLUMNS].resample('5D').mean()
# Start and end of the date range to extract
start, end = '2017-01', '2017-06'
# Plot daily and weekly resampled time series together
fig, ax = plt.subplots()
ax.plot(opsd_daily.loc[start:end, 'Wind+Solar'],
marker='.', linestyle='-', linewidth=0.5, label='Daily')
ax.plot(opsd_5d_mean.loc[start:end, 'Wind+Solar'],
marker='o', markersize=8, linestyle='-', label='5-day Mean Resample')
ax.plot(opsd_5d.loc[start:end, 'Wind+Solar'],
marker='.', linestyle='-', label='5-d Rolling Mean')
ax.set_ylabel('Solar + Wind Production (GWh)')
_ = ax.legend()

We can see that data points in the rolling mean time series have the same spacing as the daily data, but the curve is smoother because higher frequency variability has been averaged out. In the rolling mean time series, the peaks and troughs tend to align closely with the peaks and troughs of the daily time series. In contrast, the peaks and troughs in the weekly resampled time series are less closely aligned with the daily time series, since the resampled time series is at a coarser granularity.

## Trends

Time series data often exhibit some slow, gradual variability in addition to higher frequency variability such as seasonality and noise. An easy way to visualize these trends is with rolling means at different time scales.

A rolling mean tends to smooth a time series by averaging out variations at frequencies much higher than the window size and averaging out any seasonality on a time scale equal to the window size. This allows lower-frequency variations in the data to be explored. Since our electricity consumption time series has weekly and yearly seasonality, let’s look at rolling means on those two time scales.

We’ve already computed 7-day rolling means, so now let’s compute the yearly (365-day) rolling mean of our OPSD data.

In [ ]:
# The min_periods=360 argument accounts for a few isolated missing days in the
# wind and solar production time series
opsd_365d = opsd_daily[DATA_COLUMNS].rolling(
    window=365,
    center=True,
    min_periods=360
).mean()

Let’s plot the 7-day and 365-day rolling mean electricity consumption, along with the daily time series.

In [ ]:
# Plot daily, 7-day rolling mean, and 365-day rolling mean time series
fig, ax = plt.subplots()
ax.plot(opsd_daily['Consumption'], marker='.', markersize=2, color='0.6',
linestyle='None', label='Daily')
ax.plot(opsd_5d['Consumption'], linewidth=2, label='5-d Rolling Mean')
ax.plot(opsd_365d['Consumption'], color='0.2', linewidth=3,
label='Trend (365-d Rolling Mean)')
# Set x-ticks to yearly interval and add legend and labels
ax.xaxis.set_major_locator(mdates.YearLocator())
ax.legend()
ax.set_xlabel('Year')
ax.set_ylabel('Consumption (GWh)')
ax.set_title('Trends in Electricity Consumption');

We can see that the 7-day rolling mean has smoothed out all the weekly seasonality, while preserving the yearly seasonality. The 7-day rolling mean reveals that while electricity consumption is typically higher in winter and lower in summer, there is a dramatic decrease for a few weeks every winter at the end of December and beginning of January, during the holidays.

Looking at the yearly rolling mean time series, we can see that the long-term trend in electricity consumption is pretty flat, with a couple of periods of anomalously low consumption around 2009 and 2012-2013.

Now let’s look at trends in wind and solar production:

In [ ]:
# Plot 365-day rolling mean time series of wind and solar power
fig, ax = plt.subplots()
for nm in ['Wind', 'Solar', 'Wind+Solar']:
    ax.plot(opsd_365d[nm], label=nm)
    # Set x-ticks to yearly interval, adjust y-axis limits, add legend and labels
    ax.xaxis.set_major_locator(mdates.YearLocator())
    ax.set_ylim(0, 400)
    ax.legend()
    ax.set_ylabel('Production (GWh)')
    ax.set_title('Trends in Electricity Production (365-d Rolling Means)');

We can see a small increasing trend in solar power production and a more noticeable increasing trend in wind power production, as Germany continues to expand its capacity in those sectors.

## Conclusions


We’ve learned how to wrangle, analyze, and visualize our time series data in pandas using techniques such as time-based indexing, resampling, and rolling windows. Applying these techniques to our OPSD data set, we’ve gained insights on seasonality, trends, and other interesting features of electricity consumption and production in Germany.